## Installation

We will need to install a number of libraries to start with.

In [ ]:
import networkx as nx
!pip install transformers
!pip install sentencepiece
!pip install accelerate

In [ ]:
!pip install huggingface_hub
from huggingface_hub import login
access_token_write = "YOUR_HUGGINGFACE_TOKEN"  # Replace with your Hugging Face token
login(token = access_token_write)

We use flan alpaca model for speed and local execution.

In [ ]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0) 
model = AutoModelForCausalLM.from_pretrained( 
    "meta-llama/Llama-3.2-3B-Instruct",  
    device_map="auto",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct") 
pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 


model = pipeline(model="meta-llama/Llama-3.2-3B-Instruct", device=0)

def generate(messages):
    output = pipe(messages, **generation_args) 
    return output[0]['generated_text']

messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant that helps."}, 
    {"role": "user", "content": "Write an email about an alpaca that likes flan"}, 
] 
result = generate(messages)
print(result)

## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [4]:
import random

# --- 1. Distribution MBTI ---
mbti_distribution = [
    ("ISFJ", 13),
    ("ISTJ", 12.5),
    ("ESFJ", 11),
    ("ESTJ", 9),
    ("ENFP", 7),
    ("ISTP", 7),
    ("ESFP", 6.5),
    ("ISFP", 6),
    ("ESTP", 6),
]

def random_mbti():
    types, weights = zip(*mbti_distribution)
    return random.choices(types, weights=weights, k=1)[0]

In [ ]:
import networkx as nx

world_graph = nx.Graph()

prompt_meta = '''{}'''

def generate(x):
    messages = [ 
        {"role": "system", "content": "You are a person with a specific personality and past. You will be given information about your character, and perform actions as instructed."}, 
        {"role": "user", "content": x}, 
    ] 
    output = pipe(messages, **generation_args) 
    return output[0]['generated_text']


town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }


import random

class Person:
    def __init__(self, name, description):
        self.name = name
        self.description = description
        self.mbti = random_mbti()  # Randomly assign an MBTI type
        self.location = "Phandalin Town Square"
        self.memories = []
        self.plans = []
        # New economic attributes
        self.wealth = random.randint(50, 150)  # Initial wealth
        self.assets = []       # List of investments or businesses
        self.debt = 0          # Total debt
     
    def decide_next_action(self):
        # Existing method to generate an MBTI-based action (unchanged)
        context = (
            f"You are {self.name} located at {self.location}. Your MBTI personality type is {self.mbti}. "
            f"Description: {self.description}. "
        )
        prompt = (
            context +
            "Based on your personality, past experiences, and current context, what is your next action? "
            "Answer in first person, be brief, and use at most 20 words."
        )
        action = generate(prompt_meta.format(prompt))
        return action

    def decide_economic_action(self, economy, bank, market_status):
        """
        Decide on an economic action from:
         - 'consume': Spend on goods or services.
         - 'invest': Invest in an asset or project.
         - 'entrepreneur': Start a business.
         - 'borrow': Take a loan from the bank.
        
        Decision is based on MBTI type, current wealth, and market conditions.
        """
        # Base decision heuristic on wealth and MBTI type
        if self.wealth < 30:
            decision = "borrow"
        elif "N" in self.mbti:  # Intuitive types (e.g., ENFP) might lean toward investment
            decision = random.choice(["invest", "entrepreneur"])
        else:
            decision = "consume"
        
        # Execute the chosen action and update wealth
        if decision == "consume":
            amount = random.randint(5, min(20, self.wealth))
            detail = f"consume goods worth {amount}"
            self.wealth -= amount
            economy.record_transaction(buyer=self, seller=None, amount=amount, description="Consumption")
        
        elif decision == "invest":
            amount = random.randint(10, min(30, self.wealth))
            detail = f"invest {amount} in an asset"
            self.wealth -= amount
            # Record an investment with an expected return
            self.assets.append({"type": "investment", "value": amount, "expected_return": 0.1})
            economy.record_transaction(buyer=self, seller=None, amount=amount, description="Investment")
        
        elif decision == "entrepreneur":
            cost = random.randint(20, min(50, self.wealth))
            detail = f"start a business with initial investment {cost}"
            self.wealth -= cost
            # Record the creation of a business with a profit margin
            self.assets.append({"type": "business", "investment": cost, "profit_margin": 0.2})
            economy.record_transaction(buyer=self, seller=None, amount=cost, description="Entrepreneurship")
        
        elif decision == "borrow":
            amount = random.randint(20, 50)
            detail = f"borrow {amount} from bank"
            bank.provide_loan(self, amount)
            self.debt += amount
            economy.record_transaction(buyer=self, seller=None, amount=amount, description="Borrowing")
        
        else:
            detail = "no economic action"
        
        print(f"[Economic Decision] {self.name} ({self.mbti}) decides to {detail}. New wealth: {self.wealth}.")
        return decision, detail

    def __repr__(self):
        return f"{self.name} ({self.mbti}): {self.description} | Wealth: {self.wealth}"


class Economy:
    """
    Represents the closed-world economy.
    Records all economic transactions and can simulate market updates.
    """
    def __init__(self):
        self.transactions = []  # List of transaction dicts
        # Example market conditions (can be extended)
        self.market_conditions = {"consumption_demand": 1.0, "investment_rate": 1.0}
    
    def record_transaction(self, buyer, seller, amount, description):
        transaction = {
            "buyer": buyer.name,
            "seller": seller.name if seller is not None else "Market",
            "amount": amount,
            "description": description
        }
        self.transactions.append(transaction)
        print(f"[Economy] Recorded transaction: {transaction}")
    
    def update_market(self):
        # You could add logic here to evolve market conditions
        # e.g., adjust rates based on transaction volumes, etc.
        print(f"[Economy] Market update: {self.market_conditions}")


class Government:
    """
    Observer government: cannot intervene directly
    but can monitor and report on economic activity.
    """
    def __init__(self):
        self.monitored_transactions = []
    
    def monitor(self, economy):
        self.monitored_transactions = economy.transactions.copy()
    
    def report(self):
        print(f"[Government] Report: Total monitored transactions: {len(self.monitored_transactions)}")


class Bank:
    """
    Manages loans and repayments.
    """
    def __init__(self, interest_rate=0.05):
        self.loans = []  # List of loan dicts
        self.interest_rate = interest_rate
    
    def provide_loan(self, agent, amount):
        loan = {
            "agent": agent,
            "amount": amount,
            "interest_rate": self.interest_rate,
            "repaid": 0
        }
        self.loans.append(loan)
        # Loan increases the agent's wealth
        agent.wealth += amount
        print(f"[Bank] Loan of {amount} granted to {agent.name} at rate {self.interest_rate}.")
    
    def process_repayment(self, agent, amount):
        # Find the first outstanding loan for this agent
        for loan in self.loans:
            if loan["agent"] == agent and loan["amount"] > loan["repaid"]:
                remaining = loan["amount"] - loan["repaid"]
                repayment = min(amount, remaining)
                loan["repaid"] += repayment
                agent.wealth -= repayment
                print(f"[Bank] {agent.name} repaid {repayment}.")
                break

    
town_people = {
    "Toblen Stonehill": Person("Toblen Stonehill", "Toblen owns a trading post."),
    "Daran Edermath": Person("Daran Edermath", "Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard."),
    "Linene Graywind": Person("Linene Graywind", "Linene runs a trading post."),
    "Halia Thornton": Person("Halia Thornton", "Halia is an ambitious and calculating human woman."),
    "Qelline Alderleaf": Person("Qelline Alderleaf", "Qelline is a wise female halfling farmer."),
    "Sister Garaele": Person("Sister Garaele", "Sister Garaele is an elf cleric of Tymora and a Harper agent."),
    "Harbin Wester": Person("Harbin Wester", "Harbin is the townmaster of Phandalin."),
    "Terrill Bloodscar": Person("Terrill Bloodscar", "Terrill is a human ruffian and member of the Redbrands."),
    "Conrad Scarface": Person("Conrad Scarface", "Conrad is a human ruffian and member of the Redbrands."),
    "Nellie Starsmith": Person("Nellie Starsmith", "Nellie is a human ruffian and member of the Redbrands."),
    "Valerie Grinblade": Person("Valerie Grinblade", "Valerie is a human ruffian and member of the Redbrands.")
}

for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square")
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)

for person in town_people.values():
    print(person)

economy = Economy()
government = Government()
bank = Bank(interest_rate=0.05)

In [6]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [ ]:
for name in town_people.keys():
  prompt = "You are {}. {} You just woke up in the town of Phandalin and went out to the Town Square. The following people live in the town: {}. What is your goal for today? Be brief, and use at most 20 words and answer from your perspective.".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

In [8]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    prompt = "You are {}. {} Your MBTI personality type is {}. You are planning to: {}. You are currently in {} with the following description: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, town_people[name], town_people[name].mbti, plans[name], town_people[name].wealth, location,  town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i + ': ' + str(town_people[i]))
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain. You can only do something in your current location. If you want to interact with a specific person (except for yourself), respond with 'interact(x, y)', where x is the person's name as a Python string, and y is what the interaction will entail."
    action_prompts[name] = prompt

In [ ]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  print(action_results[name])
  # Now clean the action
  prompt = """
  Convert the following paragraph to first person past tense:
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

Collect the memories people observe.

In [10]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)
  
  for name in people:
    for name_two in people:
      memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [11]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [ ]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this. Only respond with the rating.".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

# Compress Memories

In [ ]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name], 
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one short sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [ ]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour. Only respond with the rating.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name], 
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [ ]:
# Nombre maximal de souvenirs à conserver pour chaque agent
MAX_MEMORIES = 50

TIME_STEPS = 100  # Exemple : simulation sur 100 itérations

for repeats in range(TIME_STEPS):
  global_time += 1
  action_prompts = {}
  # Construction des prompts d'action pour chaque zone
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(
        name,
        plans[name],
        location,
        town_areas[location],
        '\n'.join(compressed_memories_all[name][-5:]),
        str(global_time),
        ', '.join(people)
      )
      people_description = []
      for i in people:
        people_description.append(i + ': ' + str(town_people[i]))
      prompt += ' You know the following about people: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "What do you do in the next hour? Use at most 10 words to explain. You can only do something in your current location. If you want to interact with a specific person (except for yourself), respond with 'interact(x, y)', where x is the person's name as a Python string, and y is what the interaction will entail."
      action_prompts[name] = prompt

  # Génération des actions sociales MBTI
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Conversion en première personne passé (nettoyage)
    prompt_clean = """
    Convert the following paragraph to first person past tense:
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt_clean)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  
  # Conversion en tuple (Action, Object) pour représentation emoji
  action_emojis = {}
  for name in town_people.keys():
    prompt_emoji = """
    Convert the following paragraph to a tuple (Action, Object):
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt_emoji)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  
  # ----- Phase économique : chaque agent prend sa décision économique -----
  print("\nPhase économique:")
  for name in town_people.keys():
    econ_decision, econ_detail = town_people[name].decide_economic_action(economy, bank, economy.market_conditions)
    print(f"[Economic] {name} decision: {econ_decision} ({econ_detail})")
    # Ajout de l'action économique aux mémoires de l'agent
    memories[name].append('[Economic Action at {}:00: {}]'.format(str(global_time), econ_detail))
  
  # Mise à jour des mémoires sociales (observations des actions des autres)
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    for name in people:
      for name_two in people:
        memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))
  
  # Notation des mémoires
  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt_rating = "You are {}. Your plans are: {}. Your memories are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this. Only respond with the rating.".format(
        name,
        plans[name],
        '\n'.join(compressed_memories_all[name][-5:]),
        locations[name],
        str(global_time),
        memory
      )
      res = generate(prompt_meta.format(prompt_rating))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt < max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))
  
  # Compression des souvenirs importants
  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(memory_ratings[name], key=lambda x: x[1])[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt_compress = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(
      name,
      plans[name],
      locations[name],
      str(global_time),
      memory_string_to_compress
    )
    res = generate(prompt_meta.format(prompt_compress))
    compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])
  
  # Évaluation des probabilités de déplacement (place ratings)
  place_ratings = {}
  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt_place = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour. Only respond with the rating.".format(
        name,
        plans[name],
        locations[name],
        str(global_time),
        compressed_memories[name],
        area
      )
      res = generate(prompt_meta.format(prompt_place))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt < max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(place_ratings[name], key=lambda x: x[1])[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]
  
  # --- PURGE DES MÉMOIRES ANCIENNES POUR LIBÉRER LA MÉMOIRE ---
  for name in town_people.keys():
      if len(memories[name]) > MAX_MEMORIES:
          memories[name] = memories[name][-MAX_MEMORIES:]
      if len(compressed_memories_all[name]) > MAX_MEMORIES:
          compressed_memories_all[name] = compressed_memories_all[name][-MAX_MEMORIES:]
